In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip


In [2]:
import nltk
nltk.download('popular', quiet=True)

import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Dense, SpatialDropout1D, Dropout, Bidirectional
from keras.initializers import Constant
from keras.optimizers import Adam

[nltk_data] Error loading popular: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


Using TensorFlow backend.


In [3]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
submission = pd.read_csv("../input/quora-insincere-questions-classification/sample_submission.csv")

print('There are {} rows and {} columns in train'.format(train_df.shape[0],train_df.shape[1]))
print('There are {} rows and {} columns in test'.format(test_df.shape[0],test_df.shape[1]))

There are 1306122 rows and 3 columns in train
There are 375806 rows and 2 columns in test


In [4]:
majority_df = train_df[train_df['target'] == 0]
minority_df = train_df[train_df['target'] == 1]

from sklearn.utils import resample

majority_df_downsampled = resample(majority_df, 
                                 replace=False,    # sample without replacement
                                 n_samples=int(0.5*majority_df.shape[0]),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
train_df = pd.concat([majority_df_downsampled, minority_df])
train_df.shape

(693466, 3)

In [5]:
bincount = train_df.target.value_counts().to_numpy()
n_samples = train_df.target.value_counts().sum()
n_classes=2

class_weights = n_samples/(n_classes*bincount)
class_weight_dict = dict(zip(range(2), class_weights))

print(class_weight_dict)

{0: 0.5659505497375362, 1: 4.290718970424453}


In [6]:
combined_df = pd.concat([train_df,test_df])
print(combined_df.shape)

(1069272, 3)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
# from nltk import pos_tag, word_tokenize

# wordnet_lemmatizer = WordNetLemmatizer()

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'url',text)

def remove_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

# def lemmatize(text):
#     sentence_words = text.split(" ")
    
#     """
#     l = []
#     for word, tag in pos_tag(word_tokenize(text)):
#         wntag = tag[0].lower()
#         wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None

#         if not wntag:
#             lemma = word
#         else:
#             lemma = wordnet_lemmatizer.lemmatize(word, wntag)

#         l.append(lemma)

#     lemmatized = " ".join(l)
#     """
#     lemmatized = " ".join([wordnet_lemmatizer.lemmatize(w) for w in sentence_words])    
#     return lemmatized

def lower(text):
    words = text.split(" ")
    lower = " ".join([w.lower() for w in words])    
    return lower

combined_df['question_text'] = combined_df['question_text'].apply(remove_URL)
combined_df['question_text'] = combined_df['question_text'].apply(remove_punct)
# combined_df['question_text'] = combined_df['question_text'].apply(lemmatize)

In [9]:
def create_corpus_new(df):
    corpus=[]
    for text in tqdm(df['question_text']):
        words=[word.lower() for word in word_tokenize(text)]
        words=' '.join(words)
        corpus.append(words)
    return corpus   

corpus=create_corpus_new(combined_df)

100%|██████████| 1069272/1069272 [04:15<00:00, 4185.14it/s]


In [10]:
import io
import zipfile

dim=300
embedding_dict={}

with zipfile.ZipFile("../input/quora-insincere-questions-classification/embeddings.zip") as zf:
    with io.TextIOWrapper(zf.open("glove.840B.300d/glove.840B.300d.txt"), encoding="utf-8") as f:
        for line in f:
            values=line.split(' ') # ".split(' ')" only for glove-840b-300d; for all other files, ".split()" works
            word=values[0]
            vectors=np.asarray(values[1:],'float32')
            embedding_dict[word]=vectors

print(len(embedding_dict))

2196016


In [11]:
MAX_LEN = 55

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(list(corpus))
sequences=tokenizer_obj.texts_to_sequences(corpus)
text_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
word_index=tokenizer_obj.word_index

print('Number of unique words:',len(word_index))
num_words=len(word_index)+1

embedding_matrix=np.zeros((num_words,dim))
unknown_words = np.random.uniform(-1,1,size=dim).astype('float32')
unknown_words = unknown_words.reshape(1,dim)

for word, i in tqdm(word_index.items()):    
    if i > num_words:
        continue
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
    else:
        embedding_matrix[i]=unknown_words

  7%|▋         | 15783/222401 [00:00<00:01, 157823.75it/s]

Number of unique words: 222401


100%|██████████| 222401/222401 [00:01<00:00, 162912.98it/s]


In [12]:
text_pad.shape

(1069272, 55)

In [13]:
np.save('embedding_v11.npy', embedding_matrix)
np.save('text_pad_v11.npy', text_pad)

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

train=text_pad[:train_df.shape[0]]
test=text_pad[train_df.shape[0]:]

X_train,X_test,y_train,y_test=train_test_split(train,train_df['target'].values,test_size=0.2,stratify=train_df['target'].values,random_state=40)

print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (554772, 55)
Shape of Validation  (138694, 55)


In [15]:
import keras.backend as K
from sklearn.metrics import f1_score

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
# from keras.optimizers import Adam
# model=Sequential()

# embedding=Embedding(num_words,dim,embeddings_initializer=Constant(embedding_matrix),
#                    input_length=MAX_LEN,trainable=False)

# model.add(embedding)
# model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.15)))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',f1_m])
# print(model.summary())

In [17]:
from keras.layers import Flatten, Input, concatenate, GlobalMaxPooling1D
from keras.models import Model

vec_input = Input(shape=(MAX_LEN,))
embedding = Embedding(num_words,dim,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)(vec_input)
bidir1 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.15, return_sequences=True))(embedding)
bidir2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.15, return_sequences=True))(bidir1)

x = concatenate([bidir1, bidir2])
x = GlobalMaxPooling1D()(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vec_input, outputs=output)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',f1_m])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 55)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 55, 300)      66720600    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 55, 256)      439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 55, 256)      394240      bidirectional_1[0][0]            
____________________________________________________________________________________________

In [18]:
epochs=5
batch_size=128
history=model.fit(X_train,y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(X_test,y_test),
                  class_weight=class_weight_dict)

Train on 554772 samples, validate on 138694 samples
Epoch 1/5
554772/554772 [==============================] - 2921s 5ms/step - loss: 0.2753 - accuracy: 0.8838 - f1_m: 0.6407 - val_loss: 0.2630 - val_accuracy: 0.8959 - val_f1_m: 0.6657
Epoch 2/5
554772/554772 [==============================] - 2941s 5ms/step - loss: 0.2386 - accuracy: 0.9006 - f1_m: 0.6782 - val_loss: 0.2430 - val_accuracy: 0.9027 - val_f1_m: 0.6816
Epoch 3/5
554772/554772 [==============================] - 3003s 5ms/step - loss: 0.2199 - accuracy: 0.9076 - f1_m: 0.6953 - val_loss: 0.1876 - val_accuracy: 0.9261 - val_f1_m: 0.7300
Epoch 4/5
554772/554772 [==============================] - 2967s 5ms/step - loss: 0.2048 - accuracy: 0.9124 - f1_m: 0.7081 - val_loss: 0.2119 - val_accuracy: 0.9160 - val_f1_m: 0.7101
Epoch 5/5
554772/554772 [==============================] - 2943s 5ms/step - loss: 0.1917 - accuracy: 0.9170 - f1_m: 0.7205 - val_loss: 0.2192 - val_accuracy: 0.9122 - val_f1_m: 0.7028


In [19]:
model.save_weights('LSTM-bidirectional-128_32_32_1-bsize-128-epoch5.h5')

In [20]:
train_pred_GloVe = model.predict(test)
train_pred_GloVe_int = train_pred_GloVe.round().astype('int')
submission['prediction'] = train_pred_GloVe_int

In [21]:
submission.to_csv("submission.csv", index=False, header=True)